# Lineup Generator Testing

Test schedule generation with requests.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import date, timedelta
from alma_tv.scheduler.lineup import LineupGenerator
from alma_tv.scheduler.parser import RequestParser
from alma_tv.database import get_db, Session
from unittest.mock import patch
from pathlib import Path
import shutil

# Ensure we have test data
from alma_tv.library.scanner import Scanner
temp_media = Path("temp_test_media")
if not temp_media.exists():
    temp_media.mkdir()
    (temp_media / "Bluey").mkdir()
    for i in range(1, 6):
        (temp_media / "Bluey" / f"Bluey_S01E{i:02d}_Episode{i}.mp4").touch()
    (temp_media / "Peppa_Pig").mkdir()
    for i in range(1, 4):
        (temp_media / "Peppa_Pig" / f"PeppaPig_S01E{i:02d}_Episode{i}.mp4").touch()
    
    scanner = Scanner(media_root=temp_media)
    with patch.object(scanner, 'get_duration', return_value=420):
        scanner.scan_directory()


## Parse Request

In [ ]:
parser = RequestParser()
text = "tomorrow one blueie"
offset, requests = parser.parse(text)
print(f"Parsed request: {requests}")
print(f"Target date offset: {offset} days")


## Generate Lineup

In [ ]:
generator = LineupGenerator(seed=42)
target_date = date.today() + timedelta(days=offset)

session_id = generator.generate_lineup(
    target_date=target_date,
    request_payload={"requests": requests}
)

print(f"Generated Session ID: {session_id}")


## Inspect Generated Lineup

In [ ]:
if session_id:
    with get_db() as db:
        session = db.query(Session).filter(Session.id == session_id).first()
        print(f"Session for {session.show_date}")
        print(f"Status: {session.status}")
        print(f"Total duration: {session.total_duration_seconds}s")
        print("\nLineup:")
        for ph in session.play_history:
            print(f"  {ph.slot_order}. {ph.video.series} {ph.video.episode_code}")


## Test Without Request

In [ ]:
# Generate a lineup without specific requests
session_id_auto = generator.generate_lineup(
    target_date=date.today() + timedelta(days=2),
    min_episodes=3,
    max_episodes=5
)

if session_id_auto:
    with get_db() as db:
        session = db.query(Session).filter(Session.id == session_id_auto).first()
        print(f"\nAuto-generated lineup for {session.show_date}:")
        for ph in session.play_history:
            print(f"  {ph.slot_order}. {ph.video.series} {ph.video.episode_code}")
